#### Motivation
* Create feature layer of businesses by joining san diego biz directory listing and businesses from ca_business (google scraped)

In [1]:
import pandas as pd
import geopandas
import matplotlib.pyplot as plt
import sqlalchemy as sal
import psycopg2
import arcgis
from arcgis.gis import GIS
from arcgis.features import FeatureLayer, FeatureLayerCollection
import sys
sys.path.append('../../')
sys.path.append('../../../')
from gis_resources import san_diego_county_zips, get_lat_long
import os
from utils import get_config
from arcgis.features import GeoAccessor

In [2]:
gis = GIS("https://ucsdonline.maps.arcgis.com/home", client_id=get_config("arcgis","clientid"))

Please sign in to your GIS and paste the code that is obtained below.
If a web browser does not automatically open, please navigate to the URL below yourself instead.
Opening web browser to navigate to: https://ucsdonline.maps.arcgis.com/sharing/rest/oauth2/authorize?response_type=code&client_id=Elm5V3upnnV17Q3r&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&state=KakpbQE6P3nFAfV83aM4j2I0bNrXY2&allow_verification=false
Enter code obtained on signing in using SAML: ········


/Users/amolkale/Documents/DSE/DSE260-Capstone-Project/capstone/venv3.9/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ucsdonline.maps.arcgis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [3]:
nourish_user = get_config("nourish_db","username")
nourish_pswd = get_config("nourish_db","passkey")

In [4]:
conn = psycopg2.connect(
    host="awesome-hw.sdsc.edu",
    database="nourish",
    user=nourish_user,
    password=nourish_pswd)

In [5]:
# create a cursor
cur = conn.cursor()
        

# execute a statement
cur.execute('select * from "SD_City_Business_Directory"')

        # display the PostgreSQL database server version
sd_city_biz_dir_results = cur.fetchall()
       
	# close the communication with the PostgreSQL
cur.close()

In [6]:
sd_city_biz_dir_list = [(row[1],row[2],row[3],row[4],row[5],row[6],row[13]) for row in sd_city_biz_dir_results]

In [7]:
sd_city_biz_dataFrame = pd.DataFrame(sd_city_biz_dir_list,
              columns=("business_name","ownership_type", "address","city", "zip","state","business_type")
              );
print(f"Shape: {sd_city_biz_dataFrame.shape}")
sd_city_biz_dataFrame.head(4)

Shape: (50755, 7)


,business_name,ownership_type,address,city,zip,state,business_type
0,#1 FIFTH AVE,LLC,3845 05TH AVE,SAN DIEGO,92103-3140,CA,DRINKING PLACES (ALCOHOLIC BEVERAGES)
1,% NATURAL GOMEZ,SOLE,4356 POPLAR ST,SAN DIEGO,92105-4657,CA,ALL OTHER HEALTH & PERSONAL CARE STORES
2,& SALLY J FERNANDEZ,SOLE,17471 PLAZA CERADO SUITE 98,SAN DIEGO,92128-2285,CA,OTHER ACCOUNTING SERVICES
3,(W)RIGHT ON COMMUNICATIONS INC,SCORP,402 W BROADWAY SUITE 2700,SAN DIEGO,92101-8580,CA,PUBLIC RELATIONS AGENCIES


In [8]:
sd_city_biz_dataFrame["zip_code"] = sd_city_biz_dataFrame.zip.str.split('-').str[0]
print(f"Shape: {sd_city_biz_dataFrame.shape}")
sd_city_biz_dataFrame.head(4)

Shape: (50755, 8)


,business_name,ownership_type,address,city,zip,state,business_type,zip_code
0,#1 FIFTH AVE,LLC,3845 05TH AVE,SAN DIEGO,92103-3140,CA,DRINKING PLACES (ALCOHOLIC BEVERAGES),92103
1,% NATURAL GOMEZ,SOLE,4356 POPLAR ST,SAN DIEGO,92105-4657,CA,ALL OTHER HEALTH & PERSONAL CARE STORES,92105
2,& SALLY J FERNANDEZ,SOLE,17471 PLAZA CERADO SUITE 98,SAN DIEGO,92128-2285,CA,OTHER ACCOUNTING SERVICES,92128
3,(W)RIGHT ON COMMUNICATIONS INC,SCORP,402 W BROADWAY SUITE 2700,SAN DIEGO,92101-8580,CA,PUBLIC RELATIONS AGENCIES,92101


Found that we could do the address to shape conversion directly instead of converting address into lat/long.  
Refer https://developers.arcgis.com/python/api-reference/arcgis.features.toc.html#geoaccessor   
    static from_df(df, address_column='address', geocoder=None, sr=None, geometry_column=None)¶
    The from_df creates a Spatially Enabled DataFrame from a dataframe with an address column.

We might want to create full address so geocoding module works.

In [9]:
sd_city_biz_dataFrame["full_address"] = sd_city_biz_dataFrame['address'] + ', ' + sd_city_biz_dataFrame['city'] + ', ' + sd_city_biz_dataFrame['zip']+ ', ' + sd_city_biz_dataFrame['state']
sd_city_biz_dataFrame.head(2)

,business_name,ownership_type,address,city,zip,state,business_type,zip_code,full_address
0,#1 FIFTH AVE,LLC,3845 05TH AVE,SAN DIEGO,92103-3140,CA,DRINKING PLACES (ALCOHOLIC BEVERAGES),92103,"3845 05TH AVE, SAN DIEGO, 92103-3140, CA"
1,% NATURAL GOMEZ,SOLE,4356 POPLAR ST,SAN DIEGO,92105-4657,CA,ALL OTHER HEALTH & PERSONAL CARE STORES,92105,"4356 POPLAR ST, SAN DIEGO, 92105-4657, CA"


In [14]:
sd_city_biz_dataFrame.drop_duplicates(inplace=True)

In [15]:
sd_city_biz_dataFrame[['longitude','latitude']] = sd_city_biz_dataFrame.apply(lambda row: get_lat_long(row.full_address), axis='columns', result_type='expand')
print(f"shape: {sd_city_biz_dataFrame.shape}")
sd_city_biz_dataFrame.head(2)

Exception: A general error occurred: 'Response' object is not subscriptable

In [16]:
sd_city_biz_dataFrame

,business_name,ownership_type,address,city,zip,state,business_type,zip_code,full_address
0,#1 FIFTH AVE,LLC,3845 05TH AVE,SAN DIEGO,92103-3140,CA,DRINKING PLACES (ALCOHOLIC BEVERAGES),92103,"3845 05TH AVE, SAN DIEGO, 92103-3140, CA"
1,% NATURAL GOMEZ,SOLE,4356 POPLAR ST,SAN DIEGO,92105-4657,CA,ALL OTHER HEALTH & PERSONAL CARE STORES,92105,"4356 POPLAR ST, SAN DIEGO, 92105-4657, CA"
2,& SALLY J FERNANDEZ,SOLE,17471 PLAZA CERADO SUITE 98,SAN DIEGO,92128-2285,CA,OTHER ACCOUNTING SERVICES,92128,"17471 PLAZA CERADO SUITE 98, SAN DIEGO, 92128-..."
3,(W)RIGHT ON COMMUNICATIONS INC,SCORP,402 W BROADWAY SUITE 2700,SAN DIEGO,92101-8580,CA,PUBLIC RELATIONS AGENCIES,92101,"402 W BROADWAY SUITE 2700, SAN DIEGO, 92101-85..."
4,+A PARTNERS,CORP,3916 NORMAL ST,SAN DIEGO,92103-3413,CA,REAL ESTATE,92103,"3916 NORMAL ST, SAN DIEGO, 92103-3413, CA"
...,...,...,...,...,...,...,...,...,...
50750,ZZAN HILLCREST INC,SCORP,501 W UNIVERSITY AVE,SAN DIEGO,92103-3117,CA,FULL-SERVICE RESTAURANTS,92103,"501 W UNIVERSITY AVE, SAN DIEGO, 92103-3117, CA"
50751,ZZANG HOT DOG,CORP,9440 MIRA MESA BLVD,SAN DIEGO,92126-4846,CA,LIMITED-SERVICE RESTAURANTS,92126,"9440 MIRA MESA BLVD, SAN DIEGO, 92126-4846, CA"
50752,ZZANG HOTDOG AND CHICKEN UTC,CORP,4545 LA JOLLA VILLAGE DR SUITE FC3,SAN DIEGO,92122-1241,CA,FULL-SERVICE RESTAURANTS,92122,"4545 LA JOLLA VILLAGE DR SUITE FC3, SAN DIEGO,..."
50753,ZZJDESIGNS,SOLE,2358 UNIVERSITY AVE SUITE 2246,SAN DIEGO,92104-2720,CA,RETAIL SALES VIA INTERNET/WEBSITES,92104,"2358 UNIVERSITY AVE SUITE 2246, SAN DIEGO, 921..."


In [ ]:
# sd_city_biz_sdf = GeoAccessor.from_df(sd_city_biz_dataFrame,address_column="full_address")
# print(f"shape: {sd_city_biz_sdf.shape}")
# sd_city_biz_sdf.head(2)

In [ ]:
# %%time
# # Convert back from a SEDF into a feature layer Collection, and publishing on AGOL
# feature_layer_collection_item = sd_city_biz_sdf.spatial.to_featurelayer(title="Businesses (San Diego City Business Directory)", 
#                                                          gis=gis, 
#                                                          folder='nourish_gis',
#                                                          tags=['SanDiegoCountyBusinesses'],
#                                                         )
#feature_layer_collection_item